In [87]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from datetime import datetime

In [88]:
tweets = pd.read_json("/Users/sam/Documents/Masters/Data_Systems_Project/Locations/user_locations.json")

In [89]:
def split(x):
    x = str(x).split()[0]
    return x
tweets['created_at'] = tweets['created_at'].map(split)
tweets['created_at'] = pd.to_datetime(tweets['created_at'], format="%Y-%m-%d")

### Import protest data

In [90]:
protest = pd.read_csv('ACLEDNeatAdress.csv')
protest['protest'] = 1
protest = protest.rename(columns={"EVENT_DATE": "created_at", "state2": "state", 'place': 'city'})
protest['created_at'] = pd.to_datetime(protest['created_at'], format="%Y-%m-%d")

In [91]:
splitted=tweets["user_location"].str.split(", ", n = 1, expand = True)
tweets["state"] = splitted[1]
tweets["city"] = splitted[0]

In [92]:
protest = protest.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'LATITUDE', 'LONGITUDE', 'road'])

In [93]:
tweets['created_at'] = tweets['created_at'].astype(str)
protest['created_at'] = protest['created_at'].astype(str)

merged = pd.merge(tweets, protest,  how='left', on=['created_at', 'state', 'city'])
merged = merged.fillna(0)

In [94]:
merged = merged.groupby(['created_at', 'city']).size().reset_index(name='counts')

In [96]:
len(merged)

153660

In [58]:
merged = merged[merged.counts>1]
merged = pd.merge(merged, protest,  how='left', on=['created_at','city'])
merged = merged.drop(columns=['state'])
merged = merged.fillna(0)

In [60]:
merged['created_at'] = pd.to_datetime(merged['created_at'], format="%Y-%m-%d")
merged = merged[merged['created_at']>'2020-05-25']

In [62]:
mean = merged[['city', 'counts']].groupby('city').mean()
sd = merged[['city', 'counts']].groupby('city').std()

In [63]:
merged = pd.merge(merged, mean,  how='left', on=['city'])
merged = merged.rename(columns={"counts_y": "mean", 'counts_x': 'counts'})
merged = pd.merge(merged, sd,  how='left', on=['city'])
merged = merged.rename(columns={"counts_x": "counts", "counts_y": "std"})
merged = merged.fillna(0)

### Normalised

In [68]:
merged['norm'] = (merged['counts']-merged['mean'])/merged['std']

In [71]:
merged = merged.dropna()

In [72]:
import statsmodels.api as sm 

# defining the dependent and independent variables 
Xtrain = merged[['norm']]
ytrain = merged[['protest']]
   
# building the model and fitting the data 
log_reg = sm.Logit(ytrain, Xtrain).fit()

Optimization terminated successfully.
         Current function value: 0.693082
         Iterations 3


In [73]:
print(log_reg.summary()) 

                           Logit Regression Results                           
Dep. Variable:                protest   No. Observations:                68009
Model:                          Logit   Df Residuals:                    68008
Method:                           MLE   Df Model:                            0
Date:                Mon, 18 Jan 2021   Pseudo R-squ.:                  -2.605
Time:                        12:02:51   Log-Likelihood:                -47136.
converged:                       True   LL-Null:                       -13075.
Covariance Type:            nonrobust   LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
norm           0.0241      0.008      2.969      0.003       0.008       0.040


In [ ]:
crowd counting
number of people in protest
weekday effects
account for days on which verdicts and important new stories aired

### Fixed Effects

In [79]:
import numpy as np
import pandas as pd
from linearmodels import PanelOLS

In [84]:
merged.city =  pd.Categorical(merged.city)
merged = merged.set_index('created_at', append=True)

In [85]:
#include logistic regression

exog_vars = ['counts', 'city']
exog = sm.add_constant(merged[exog_vars])
mod = PanelOLS(merged.protest, exog)
re_res = mod.fit()
print(re_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                protest   R-squared:                        0.2145
Estimator:                   PanelOLS   R-squared (Between):              0.2145
No. Observations:               68009   R-squared (Within):               0.0000
Date:                Mon, Jan 18 2021   R-squared (Overall):              0.2145
Time:                        12:36:49   Log-likelihood                 1.671e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2.2695
Entities:                       68009   P-value                           0.0000
Avg Obs:                       1.0000   Distribution:              F(7305,60703)
Min Obs:                       1.0000                                           
Max Obs:                       1.0000   F-statistic (robust):             2.2695
                            